Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Responsible AI dashboard for Time Series Forecasting
_**Orange Juice Sales Forecasting**_

## Contents
1. [Introduction](#introduction)
1. [Data](#data)
1. [Train](#train)
1. [Forecast](#forecast)
1. [Responsible AI Dashboard](#analyze)

## Introduction<a id="introduction"></a>
In this example, we use sktime to train, select, and operationalize a time-series forecasting model for multiple time-series.

The examples in the follow code samples use the University of Chicago's Dominick's Finer Foods dataset to forecast orange juice sales. Dominick's was a grocery chain in the Chicago metropolitan area.

In [5]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer, RobustScaler, MinMaxScaler
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.forecasting.compose import make_reduction, DirectTabularRegressionForecaster
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series

## Data<a id="data"></a>
You are now ready to load the historical orange juice sales data. We will load the CSV file into a plain pandas DataFrame; the time column in the CSV is called _WeekStarting_, so it will be specially parsed into the datetime type.

In [24]:
time_column_name = "WeekStarting"
time_series_id_column_names = ["Store", "Brand"]
dataset_location = "https://raw.githubusercontent.com/Azure/azureml-examples/2fe81643865e1f4591e7734bd1a729093cafb826/v1/python-sdk/tutorials/automl-with-azureml/forecasting-orange-juice-sales/dominicks_OJ.csv"
data = pd.read_csv(dataset_location, parse_dates=[time_column_name])

# Drop the columns 'logQuantity' as it is a leaky feature.
data.drop("logQuantity", axis=1, inplace=True)

data.set_index(time_series_id_column_names + [time_column_name], inplace=True)
data.asfreq("W-THU")
data.sort_index(inplace=True, ascending=[True, True, True])

data.head()

TypeError: Cannot convert input [(2, 'dominicks', Timestamp('1990-06-14 00:00:00'))] of type <class 'tuple'> to Timestamp

Each row in the DataFrame holds a quantity of weekly sales for an OJ brand at a single store. The data also includes the sales price, a flag indicating if the OJ brand was advertised in the store that week, and some customer demographic information based on the store location. For historical reasons, the data also include the logarithm of the sales quantity. The Dominick's grocery data is commonly used to illustrate econometric modeling techniques where logarithms of quantities are generally preferred.    

The task is now to build a time-series model for the _Quantity_ column. It is important to note that this dataset is comprised of many individual time-series - one for each unique combination of _Store_ and _Brand_. To distinguish the individual time-series, we define the **time_series_id_column_names** - the columns whose values determine the boundaries between time-series: 

In [7]:
nseries = data.groupby(time_series_id_column_names).ngroups
print("Data contains {0} individual time-series.".format(nseries))

Data contains 249 individual time-series.


For demonstration purposes, we extract sales time-series for just a few of the stores:

In [14]:
use_stores = [2, 5, 8]
data_subset = data.loc[([2,5,8], slice(None), slice(None)), :]
nseries = data_subset.groupby(time_series_id_column_names).ngroups
print("Data subset contains {0} individual time-series.".format(nseries))

Data subset contains 9 individual time-series.


### Data Splitting
We now split the data into a training and a testing set for later forecast evaluation. The test set will contain the final 20 weeks of observed sales for each time-series. The splits should be stratified by series, so we use a group-by statement on the time series identifier columns.

In [15]:
target_column_name = "Quantity"

y = pd.DataFrame(data_subset[target_column_name])
X = data_subset.drop(columns=[target_column_name])
fh_dates = pd.DatetimeIndex(y.index.get_level_values(2).unique().sort_values().to_list()[-20:], freq='W-THU')
fh = ForecastingHorizon(fh_dates, is_relative=False)
y_train, y_test, X_train, X_test = \
    temporal_train_test_split(
        y=y,
        X=X,
        test_size=20)

## Modeling

For forecasting tasks, we need to do several pre-processing and estimation steps that are specific to time-series including:
* Detect time-series sample frequency (e.g. hourly, daily, weekly) and create new records for absent time points to make the series regular. A regular time series has a well-defined frequency and has a value at every sample point in a contiguous time span 
* Impute missing values in the target (via forward-fill) and feature columns (using median column values) 
* Create features based on time series identifiers to enable fixed effects across different series
* Create time-based features to assist in learning seasonal patterns
* Encode categorical variables to numeric quantities

In this notebook, we will train a single, regression-type model across **all** time-series in a given training set. This allows the model to generalize across related series.

## Customization

The featurization customization in forecasting is an advanced feature in AutoML which allows our customers to change the default forecasting featurization behaviors and column types through `FeaturizationConfig`. The supported scenarios include:

1. Column purposes update: Override feature type for the specified column. Currently supports DateTime, Categorical and Numeric. This customization can be used in the scenario that the type of the column cannot correctly reflect its purpose. Some numerical columns, for instance, can be treated as Categorical columns which need to be converted to categorical while some can be treated as epoch timestamp which need to be converted to datetime. To tell our SDK to correctly preprocess these columns, a configuration need to be add with the columns and their desired types.
2. Transformer parameters update: Currently supports parameter change for Imputer only. User can customize imputation methods. The supported imputing methods for target column are constant and ffill (forward fill). The supported imputing methods for feature columns are mean, median, most frequent, constant and ffill (forward fill). This customization can be used for the scenario that our customers know which imputation methods fit best to the input data. For instance, some datasets use NaN to represent 0 which the correct behavior should impute all the missing value with 0. To achieve this behavior, these columns need to be configured as constant imputation with `fill_value` 0.
3. Drop columns: Columns to drop from being featurized. These usually are the columns which are leaky or the columns contain no useful data.

In [ ]:
#forecaster = TransformedTargetForecaster(steps=[
#    # ("detrend", Detrender()),
#    # ("deseasonalize", Deseasonalizer()),
#    # ("minmax", TabularToSeriesAdaptor(MinMaxScaler((1, 10)))),
#    # ("power", TabularToSeriesAdaptor(PowerTransformer())),
#    # ("scaler", TabularToSeriesAdaptor(RobustScaler())),
#    ("forecaster", DirectTabularRegressionForecaster(KNeighborsTimeSeriesRegressor())),
#])

"""
featurization_config = FeaturizationConfig()
# Force the CPWVOL5 feature to be numeric type.
featurization_config.add_column_purpose("CPWVOL5", "Numeric")
# Fill missing values in the target column, Quantity, with zeros.
featurization_config.add_transformer_params(
    "Imputer", ["Quantity"], {"strategy": "constant", "fill_value": 0}
)
# Fill missing values in the INCOME column with median value.
featurization_config.add_transformer_params(
    "Imputer", ["INCOME"], {"strategy": "median"}
)
# Fill missing values in the Price column with forward fill (last value carried forward).
featurization_config.add_transformer_params("Imputer", ["Price"], {"strategy": "ffill"})
"""

## Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the ForecastingParameters class. The table below details the forecasting parameter we will be passing into our experiment.


|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**time_series_id_column_names**|The column names used to uniquely identify the time series in data that has multiple rows with the same timestamp. If the time series identifiers are not defined, the data set is assumed to be one time series.|
|**freq**|Forecast frequency. This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.
|**cv_step_size**|Number of periods between two consecutive cross-validation folds. The default value is "auto", in which case AutoMl determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value.

## Train<a id="train"></a>

You can now submit a new training run. Depending on the data and number of iterations this operation may take several minutes.
Information from each iteration will be printed to the console.  Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [18]:
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.impute import Imputer

forecaster = ForecastingPipeline(
    steps=[
        ("imputer", Imputer(method="mean")),
        ("forecaster", DirectTabularRegressionForecaster(KNeighborsTimeSeriesRegressor()))])
forecaster.fit(y=y_train, X=X_train, fh=fh)
forecaster.predict(fh=fh, X=X_test)

TypeError: unsupported operand type(s) for *=: 'int' and 'NoneType'

# Forecast<a id="forecast"></a>

Now that we have retrieved the best pipeline/model, it can be used to make predictions on test data.

### Retrieving forecasts from the model

To produce predictions on the test set, we need to know the feature values at all dates in the test set. This requirement is somewhat reasonable for the OJ sales data since the features mainly consist of price, which is usually set in advance, and customer demographics which are approximately constant for each store over the 20 week forecast horizon in the testing data.

Try downloading the model and running forecasts locally.

In [ ]:
import joblib
best_run.download_file('outputs/model.pkl')
model = joblib.load('model.pkl')
model

In [ ]:
# forecast returns the predictions for all specified quantiles
# use 0.025 and 0.975 for the 95% confidence interval
model.quantiles = [0.025, 0.5, 0.975]
y_pred_quantiles = model.forecast_quantiles(test.loc[:, test.columns != target_column_name])
y_pred_quantiles.head()

# Responsible AI Dashboard<a id="analyze"></a>

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIForecastingInsights, FeatureMetadata

feature_metadata = FeatureMetadata(
    time_series_id_column_names=time_series_id_column_names, 
    categorical_features=time_series_id_column_names,
    time_column_name=time_column_name)
insights = RAIForecastingInsights(
    model=model,
    train=train,
    test=test,
    target_column=target_column_name,
    feature_metadata=feature_metadata)

ResponsibleAIDashboard(insights)